## **Leveraging Variational Autoencoders (VAEs) for Classification of Anthropogenic Aerosols (with Outlier Analysis)**

By: Aaryan Doshi, Mentor: Dr. Lamb


Install pyroVED package:

In [ ]:
# Install the latest version from source
# (or you may install a stable version from PyPi instead with !pip install pyroved)
!pip install -q git+https://github.com/ziatdinovmax/pyroved.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.8 MB/s eta 0:00:00


In [ ]:
import os
import math
import numpy as np
import seaborn as sns
import pandas as pd
import time
import random

import torch

from torch.nn import Conv1d, ConvTranspose1d
from torch.nn import Linear as Lin, ReLU, Sigmoid, ConstantPad1d
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold

from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
import pyroved as pv
import torch
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_train = np.load("/content/drive/MyDrive/Aerosols_Proj/Lab_X_train.npy")
Y_train = np.load("/content/drive/MyDrive/Aerosols_Proj/Lab_Y_train.npy")

X_val = np.load("/content/drive/MyDrive/Aerosols_Proj/Lab_X_val.npy")
Y_val = np.load("/content/drive/MyDrive/Aerosols_Proj/Lab_Y_val.npy")

X_test = np.load("/content/drive/MyDrive/Aerosols_Proj/Lab_X_test.npy")
Y_test = np.load("/content/drive/MyDrive/Aerosols_Proj/Lab_Y_test.npy")

In [ ]:
# for reproducibility
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
X_train.shape

(140082, 400, 4)

In [ ]:
X_test.shape

(70042, 400, 4)

In [ ]:
X_val.shape

(70041, 400, 4)

In [ ]:
Y_train.shape

(140082,)

In [ ]:
import random
import matplotlib.pyplot as plt

for class_label in range(1, 7):
    class_indices = [i for i, label in enumerate(Y_train) if label == class_label]

    random_index = random.choice(class_indices)

    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(12, 3))
    axes[0].plot(X_train[random_index, :, 0])
    axes[0].set_title(f"Ch. 0 - Class {class_label}")
    axes[0].set_xticks([])
    axes[0].set_yticks([])
    axes[1].plot(X_train[random_index, :, 1])
    axes[1].set_title(f"Ch. 1 - Class {class_label}")
    axes[1].set_xticks([])
    axes[1].set_yticks([])
    axes[2].plot(X_train[random_index, :, 2])
    axes[2].set_title(f"Ch. 2 - Class {class_label}")
    axes[2].set_xticks([])
    axes[2].set_yticks([])
    axes[3].plot(X_train[random_index, :, 3])
    axes[3].set_title(f"Ch. 3 - Class {class_label}")
    axes[3].set_xticks([])
    axes[3].set_yticks([])

    plt.show()

In [ ]:
n = random.randint(1, 140081)
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(12,3))
axes[0].plot(X_train[n,:,0])
axes[0].set_title("Ch. 0")
axes[1].plot(X_train[n,:,1])
axes[1].set_title("Ch. 1")
axes[2].plot(X_train[n,:,2])
axes[2].set_title("Ch. 2")
axes[3].plot(X_train[n,:,3])
axes[3].set_title("Ch. 3")

random_indices = [random.randint(0, 140081) for _ in range(5)]

for idx in random_indices:
    print(X_train[idx, 199, :])

# print out the channel 0, 1, 2, 3, 4 for one random aerosol sample in X_train.
# this will be across a 400 point time series


In [ ]:
Y_train.shape

(140082,)

In [ ]:
# APPLY INCANDESCENT THRESHOLD
mask = np.any(X_train[:, :, 1] > 0.2, axis=1)
X_train = X_train[mask]
Y_train = Y_train[mask]

In [ ]:
import numpy as np

unique_classes_orig, class_counts_orig = np.unique(Y_train, return_counts=True)

mask = np.any(X_train[:, :, 1] > 0.2, axis=1)
X_train_filtered = X_train[mask]
Y_train_filtered = Y_train[mask]

unique_classes_after, class_counts_after = np.unique(Y_train_filtered, return_counts=True)

print("Different aerosol classes detectable incandesence:")
for cls in unique_classes_orig:
    count_orig = class_counts_orig[unique_classes_orig == cls][0]
    count_after = class_counts_after[unique_classes_after == cls][0] if cls in unique_classes_after else 0
    percentage_change = (count_after / count_orig) * 100 if count_orig > 0 else 0
    print(f"Class {cls}: Before = {count_orig}, After = {count_after}, Percentage = {percentage_change:.2f}%")

Different aerosol classes detectable incandesence:
Class 0.0: Before = 19660, After = 19660, Percentage = 100.00%
Class 1.0: Before = 16223, After = 16223, Percentage = 100.00%
Class 2.0: Before = 1706, After = 1706, Percentage = 100.00%
Class 3.0: Before = 4786, After = 4786, Percentage = 100.00%
Class 4.0: Before = 5402, After = 5402, Percentage = 100.00%
Class 5.0: Before = 18377, After = 18377, Percentage = 100.00%
Class 6.0: Before = 19678, After = 19678, Percentage = 100.00%


In [ ]:
X_train.shape

(85832, 400, 4)

In [ ]:
Y_train.shape

(85832,)

In [ ]:
original_max_X_train = np.zeros((len(X_train), 4))
for p_idx in range(len(X_train)):
  for c_idx in range(4):
    original_max_X_train[p_idx][c_idx] = np.max(X_train[p_idx, :, c_idx])


In [ ]:
X_train.shape

(85832, 400, 4)

In [ ]:
peak_height = np.max(X_train[:, :, 1])
print(peak_height)

132.61664460622106


In [ ]:
#SATURATION NUMERIC

#creating a more accurate delta eps values.

# Specify  epsilon value, like how tolerant will be to change
eps = 0.001

# Step 1: Find the maximum signal per particle in Channel 0
max_signal_channel_0 = np.max(X_train[:, :, 0], axis=1)

# Step 2: Count the number of points within epsilon of the maximum for each particle in Channel 0
# This comparison now includes the epsilon tolerance around the maximum signal
count_max_signal_channel_0 = np.sum((X_train[:, :, 0] >= (max_signal_channel_0[:, np.newaxis] - eps)) &
                                    (X_train[:, :, 0] <= (max_signal_channel_0[:, np.newaxis] + eps)), axis=1)

print(count_max_signal_channel_0/400*100)
count_max_signal_channel_0.shape


[ 0.25  0.5   0.25 ... 14.25  8.75 28.75]


(85832,)

In [ ]:
max1 = np.max(X_train[:, :, 1], axis=1)
max2 = np.max(X_train[:, :, 2], axis=1)

# Calculate the color channel ratio for all 400 points
color_channel_ratio = max2 / max1

color_channel_ratio

array([1.11002002, 1.08262394, 1.21032505, ..., 1.52508171, 1.49707908,
       1.61149563])

In [ ]:
channel_min_X_train = np.zeros((len(X_train), 4))
channel_max_X_train = np.zeros((len(X_train), 4))

# WOULD APPLY SAME LOGIC TO VALIDATION AND TEST DATASETS

for p_idx in range(len(X_train)):
  for c_idx in range(4):
    channel_min_X_train[p_idx][c_idx] = np.min(X_train[p_idx, :, c_idx])

  for c_idx in range(4):
    channel_max_X_train[p_idx][c_idx] = np.max(X_train[p_idx, :, c_idx])


for p_idx in range (0,len(X_train)):
  for c_idx in range(0, 4):
    s0 = (X_train[p_idx,:,c_idx] - channel_min_X_train[p_idx][c_idx])/(channel_max_X_train[p_idx][c_idx]- channel_min_X_train[p_idx][c_idx])
    X_train[p_idx,:,c_idx] = s0

X_train

In [ ]:
print(type(X_train))

<class 'numpy.ndarray'>


In [ ]:
X_train_C0 = torch.from_numpy(X_train[:,:, 0]).float()

train_loader_0 = pv.utils.init_dataloader(X_train_C0.unsqueeze(1), batch_size=64)

fig, axes = plt.subplots(8, 8, figsize=(8, 8), dpi=300,
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))

for ax, (y,) in zip(axes.flat, train_loader_0):
    y_use = y.numpy()
    ax.plot(y_use[0, 0])

plt.savefig('high_res_figure.png', dpi=300)
plt.close(fig)

from google.colab import files
files.download('high_res_figure.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
X_train_C1 = torch.from_numpy(X_train[:,:, 1]).float()

train_loader_1 = pv.utils.init_dataloader(X_train_C1.unsqueeze(1), batch_size=64)

fig, axes = plt.subplots(8, 8, figsize=(8, 8), dpi=300,
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))

for ax, (y,) in zip(axes.flat, train_loader_1):
    y_use = y.numpy()
    ax.plot(y_use[0, 0])

plt.savefig('high_res_figure_channel_1.png', dpi=300)
plt.close(fig)

from google.colab import files
files.download('high_res_figure_channel_1.png')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
channel_0 = (140081, )

# Initialize vanilla VAE
vae = pv.models.iVAE(data_dim = (400,), latent_dim = 2, invariances=None, sampler_d = 'gaussian', seed=0)

# Initialize SVI trainer
trainer = pv.trainers.SVItrainer(vae)

# Train for n epochs:
for e in range(55):
    trainer.step(train_loader_0)
    trainer.print_statistics()

In [ ]:
# PATH = "/content/drive/MyDrive/Aerosols_Proj/2D_VAE"
# torch.save(vae.state_dict(), PATH)

In [ ]:
# vae = pv.models.iVAE(data_dim = (400,), latent_dim = 2, invariances=None, sampler_d = 'gaussian', seed=0)
# vae.load_state_dict(torch.load(PATH, weights_only=True))
# vae.eval()

In [ ]:
# Manifold Channel 0
vae.manifold2d(d = 10)

In [ ]:
channel_1 = (140081, )

# Initialize vanilla VAE

vae = pv.models.iVAE(data_dim = (400,), latent_dim = 2, invariances=None, sampler_d = 'gaussian', seed=0)

# Initialize SVI trainer
trainer = pv.trainers.SVItrainer(vae)

# Train for n epochs:
for e in range(55):
    trainer.step(train_loader_1)
    trainer.print_statistics()

**ANALYZING OUTLIERS + Encode the training data into the latent space:**

In [ ]:
# Manifold Channel 1
vae.manifold2d(d = 10)

In [ ]:
# 2D PLOTS (NON-ADAPTIVE VERSION) --> CHANNEL 0

# ANALYZING OUTLIERS
from scipy.spatial.distance import cdist
from matplotlib.colors import ListedColormap

global outlier_indices

def identify_outliers_with_percentages(latent_space, labels, z_threshold=3.0):
    unique_labels = np.unique(labels)
    outliers = []
    all_distances = []
    outlier_indices = []
    # to implement outlier percentage by class
    outlier_percentages = {}

    for label in unique_labels:
        cluster_points = latent_space[labels == label]
        total_points = len(cluster_points)
        centroid = cluster_points.mean(axis=0)
        distances = cdist(cluster_points, [centroid], metric='euclidean').flatten()
        all_distances.extend(distances)
        mean_distance = distances.mean()
        std_distance = distances.std()
        threshold = mean_distance + z_threshold * std_distance
        cluster_outliers = cluster_points[distances > threshold]

        # calculating % of outliers per class (fly ash, dust, etc.)
        # per class basis
        num_outliers = len(cluster_outliers)
        outlier_percent = (num_outliers / total_points) * 100 if total_points > 0 else 0
        outlier_percentages[label] = outlier_percent

        outliers.append(cluster_outliers)
        outlier_indices.extend(np.where(labels == label)[0][distances > threshold])

    if outliers:
        all_outliers = np.vstack(outliers)
    else:
        all_outliers = np.array([]).reshape(0, latent_space.shape[1])

    return all_outliers, all_distances, outlier_indices, outlier_percentages

def plot_latent_space_with_threshold(z_threshold):
    # Use updated function to calculate outliers and percentages
    outliers, all_distances, outlier_indices, outlier_percentages = identify_outliers_with_percentages(z_mean, Y_train, z_threshold)

    # Print percentage of outliers for each class
    for label, percentage in outlier_percentages.items():
        print(f"Class {label}: {percentage:.2f}% of points classified as outliers.")

    non_outlier_mask = np.ones(len(z_mean), dtype=bool)
    non_outlier_mask[outlier_indices] = False

    z_mean_filtered = z_mean[non_outlier_mask]
    Y_train_filtered = Y_train[non_outlier_mask]

    # removing atd, cfa, va
    classes_to_remove = [3, 2, 4]
    class_mask = ~np.isin(Y_train_filtered, classes_to_remove)

    z_mean_class_filtered = z_mean_filtered[class_mask]
    Y_train_class_filtered = Y_train_filtered[class_mask]

    fig, axes = plt.subplots(1, 4, figsize=(60, 12), dpi=300)

    target_names = ["FS", "FSglyc", "CFA", "ATD", "VA", "Fe2O3", "Fe3O4"]
    cmap1 = ListedColormap(['red', 'purple', 'yellow', 'gray', 'brown', 'blue', 'green'])

    # trying to plot the original latent space
    im1 = axes[0].scatter(z_mean[:, -1], z_mean[:, -2], s=1, c=Y_train, cmap=cmap1, label='Data Points')
    axes[0].set_title(f"Original Latent Space (z_threshold = {z_threshold})", fontsize=21)
    axes[0].set_xlabel("$z_1$", fontsize=21)
    axes[0].set_ylabel("$z_2$", fontsize=21)
    axes[0].legend(fontsize=21)
    axes[0].tick_params(axis='both', which='major', labelsize=21)
    cbar1 = fig.colorbar(im1, ax=axes[0], ticks=range(len(cmap1.colors)), shrink=.8)
    cbar1.set_ticklabels(target_names, fontsize=21)
    cbar1.set_label("$\mu$", fontsize=21)

    # trying to plot the latent space without outliers
    im2 = axes[1].scatter(z_mean_filtered[:, -1], z_mean_filtered[:, -2], s=1, c=Y_train_filtered, cmap=cmap1, label='Data Points')
    axes[1].set_title("Latent Space without Outliers", fontsize=21)
    axes[1].set_xlabel("$z_1$", fontsize=21)
    axes[1].set_ylabel("$z_2$", fontsize=21)
    axes[1].legend(fontsize=21)
    axes[1].tick_params(axis='both', which='major', labelsize=21)
    cbar2 = fig.colorbar(im2, ax=axes[1], ticks=range(len(cmap1.colors)), shrink=.8)
    cbar2.set_ticklabels(target_names, fontsize=21)
    cbar2.set_label("$\mu$", fontsize=21)

    # trying to plot the outliers specifically
    im3 = axes[2].scatter(z_mean[:, -1], z_mean[:, -2], s=1, c=Y_train, cmap=cmap1, label='Data Points')
    axes[2].scatter(outliers[:, -1], outliers[:, -2], s=10, c='black', marker='x', label='Outliers')
    axes[2].set_title("Outliers in Latent Space", fontsize=21)
    axes[2].set_xlabel("$z_1$", fontsize=21)
    axes[2].set_ylabel("$z_2$", fontsize=21)
    axes[2].legend(fontsize=21)
    axes[2].tick_params(axis='both', which='major', labelsize=21)
    cbar3 = fig.colorbar(im3, ax=axes[2], ticks=range(len(cmap1.colors)), shrink=.8)
    cbar3.set_ticklabels(target_names, fontsize=21)
    cbar3.set_label("$\mu$", fontsize=21)

    # this one to have latent space wo specific classes
    im4 = axes[3].scatter(z_mean_class_filtered[:, -1], z_mean_class_filtered[:, -2], s=1, c=Y_train_class_filtered, cmap=cmap1, label='Data Points')
    axes[3].set_title("Latent Space without ATD, CFA, and VA", fontsize=21)
    axes[3].set_xlabel("$z_1$", fontsize=21)
    axes[3].set_ylabel("$z_2$", fontsize=21)
    axes[2].legend(fontsize=21)
    axes[2].tick_params(axis='both', which='major', labelsize=21)
    cbar4 = fig.colorbar(im4, ax=axes[3], ticks=range(len(cmap1.colors)), shrink=.8)
    cbar4.set_ticklabels(target_names, fontsize=21)
    cbar4.set_label("$\mu$", fontsize=21)

z_mean, z_sd = vae.encode(X_train_C0)

plot_latent_space_with_threshold(0.0001)
plot_latent_space_with_threshold(2.0)

plt.show()


In [ ]:
# 2D PLOTS (NON-ADAPTIVE VERSION) --> CHANNEL 1

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from matplotlib.colors import ListedColormap

def identify_outliers_with_percentages(latent_space, labels, z_threshold=3.0):
    unique_labels = np.unique(labels)
    outliers = []
    all_distances = []
    outlier_indices = []
    outlier_percentages = {}

    for label in unique_labels:
        cluster_points = latent_space[labels == label]
        total_points = len(cluster_points)  # Total points in the cluster
        global_centroid = cluster_points.mean(axis=0)
        distances = cdist(cluster_points, [global_centroid], metric='euclidean').flatten()
        all_distances.extend(distances)
        mean_distance = distances.mean()
        std_distance = distances.std()
        threshold = mean_distance + z_threshold * std_distance
        cluster_outliers = cluster_points[distances > threshold]

        # calculating % of outliers per class (fly ash, dust, etc.)
        # per class basis
        num_outliers = len(cluster_outliers)
        outlier_percent = (num_outliers / total_points) * 100 if total_points > 0 else 0
        outlier_percentages[label] = outlier_percent

        outliers.append(cluster_outliers)
        outlier_indices.extend(np.where(labels == label)[0][distances > threshold])



    if outliers:
        all_outliers = np.vstack(outliers)
    else:
        all_outliers = np.array([]).reshape(0, latent_space.shape[1])

    return all_outliers, all_distances, outlier_indices, outlier_percentages

def plot_latent_space_with_threshold(z_threshold):
    # Use updated function to calculate outliers and percentages
    outliers, all_distances, outlier_indices, outlier_percentages = identify_outliers_with_percentages(z_mean_1, Y_train, z_threshold)

    # Print percentage of outliers for each class
    for label, percentage in outlier_percentages.items():
        print(f"Class {label}: {percentage:.2f}% of points classified as outliers.")

    non_outlier_mask = np.ones(len(z_mean_1), dtype=bool)
    non_outlier_mask[outlier_indices] = False

    z_mean_filtered = z_mean_1[non_outlier_mask]
    Y_train_filtered = Y_train[non_outlier_mask]

    classes_to_remove = [3, 2, 4]
    class_mask = ~np.isin(Y_train_filtered, classes_to_remove)

    z_mean_class_filtered = z_mean_filtered[class_mask]
    Y_train_class_filtered = Y_train_filtered[class_mask]

    fig, axs = plt.subplots(1, 3, figsize=(24, 8), dpi=300)

    target_names = ["FS", "FSglyc", "CFA", "ATD", "VA", "Fe2O3", "Fe3O4"]
    cmap1 = ListedColormap(['red', 'purple', 'yellow', 'gray', 'brown', 'blue', 'green'])

    font_size = 12
    colorbar_shrink = 0.5

    # Plot original latent space
    im1 = axs[0].scatter(z_mean_1[:, -1], z_mean_1[:, -2], s=1, c=Y_train, cmap=cmap1)
    axs[0].set_title(f"Original Latent Space (z_threshold = {z_threshold})", fontsize=font_size)
    axs[0].set_xlabel("$z_2$", fontsize=font_size)
    axs[0].set_ylabel("$z_3$", fontsize=font_size)
    cbar1 = fig.colorbar(im1, ax=axs[0], ticks=range(len(cmap1.colors)), shrink=colorbar_shrink)
    cbar1.set_ticklabels(target_names)
    cbar1.set_label("$\mu$", fontsize=font_size)

    # Plot latent space without outliers
    im2 = axs[1].scatter(z_mean_filtered[:, -1], z_mean_filtered[:, -2], s=1, c=Y_train_filtered, cmap=cmap1)
    axs[1].set_title("Latent Space without Outliers", fontsize=font_size)
    axs[1].set_xlabel("$z_2$", fontsize=font_size)
    axs[1].set_ylabel("$z_3$", fontsize=font_size)
    cbar2 = fig.colorbar(im2, ax=axs[1], ticks=range(len(cmap1.colors)), shrink=colorbar_shrink)
    cbar2.set_ticklabels(target_names)
    cbar2.set_label("$\mu$", fontsize=font_size)

    # Plot the outliers specifically
    im3 = axs[2].scatter(z_mean_1[:, -2], z_mean_1[:, -1], s=1, c=Y_train, cmap=cmap1)
    axs[2].scatter(outliers[:, -2], outliers[:, -1], s=10, c='black', marker='x', label='Outliers')
    axs[2].set_title("Outliers in Latent Space", fontsize=font_size)
    axs[2].set_xlabel("$z_2$", fontsize=font_size)
    axs[2].set_ylabel("$z_3$", fontsize=font_size)
    cbar3 = fig.colorbar(im3, ax=axs[2], ticks=range(len(cmap1.colors)), shrink=colorbar_shrink)
    cbar3.set_ticklabels(target_names)
    cbar3.set_label("$\mu$", fontsize=font_size)
    axs[2].legend(fontsize=font_size)

    plt.show()

z_mean_1, z_sd_1 = vae.encode(X_train_C1)

plot_latent_space_with_threshold(3.0)
plot_latent_space_with_threshold(2.0)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from matplotlib.colors import ListedColormap
from matplotlib.patches import ConnectionPatch

def identify_outliers_with_percentages(latent_space, labels, z_threshold=3.0):
    unique_labels = np.unique(labels)
    outliers = []
    outlier_indices = []

    for label in unique_labels:
        cluster_points = latent_space[labels == label]
        global_centroid = cluster_points.mean(axis=0)
        distances = cdist(cluster_points, [global_centroid], metric='euclidean').flatten()

        mean_distance = distances.mean()
        std_distance = distances.std()
        threshold = mean_distance + z_threshold * std_distance
        cluster_outliers = cluster_points[distances > threshold]

        outliers.append(cluster_outliers)
        outlier_indices.extend(np.where(labels == label)[0][distances > threshold])

    return np.vstack(outliers) if outliers else np.array([]), outlier_indices

def plot_latent_space_with_signal(z_threshold, chosen_outlier_idx):
    target_names = ["FS", "FSglyc", "CFA", "ATD", "VA", "Fe2O3", "Fe3O4"]
    outliers, outlier_indices = identify_outliers_with_percentages(z_mean_1, Y_train, z_threshold)

    fig = plt.figure(figsize=(14, 9), dpi=200)
    gs = fig.add_gridspec(2, 2, width_ratios=[2, 1], height_ratios=[1, 1])

    ax = fig.add_subplot(gs[:, 0])
    cmap1 = ListedColormap(['red', 'purple', 'yellow', 'gray', 'brown', 'blue', 'green'])

    ax.scatter(z_mean_1[:, -1], z_mean_1[:, -2], s=1, c=Y_train, cmap=cmap1)
    ax.scatter(outliers[:, -1], outliers[:, -2], s=10, c='black', marker='x', label='Outliers')

    chosen_outlier = z_mean_1[chosen_outlier_idx]
    chosen_class = int(Y_train[chosen_outlier_idx])
    chosen_color = cmap1.colors[chosen_class]
    chosen_class_name = target_names[chosen_class]

    ax.set_title(f"Latent Space with Outliers - Selected: {chosen_class_name} (z_threshold = {z_threshold})")
    ax.set_xlabel("$z_3$")
    ax.set_ylabel("$z_4$")

    ax.scatter([], [], s=30, c=[chosen_color], marker='o', edgecolor='black', linewidth=1.2,
               label=f'Selected Outlier ({chosen_class_name})')
    ax.legend()

    ax_signal = fig.add_subplot(gs[0, 1])
    for ch in range(4):
        ax_signal.plot(X_train[chosen_outlier_idx, :, ch], label=f"Ch. {ch}")

    ax_signal.set_title(f"Signal of Selected Outlier ({chosen_class_name})")
    ax_signal.legend(fontsize=8)
    ax_signal.set_xlabel("Time")
    ax_signal.set_ylabel("Amplitude")

    gs_bottom = gs[1, 1].subgridspec(2, 2, wspace=0.5, hspace=0.5)
    ax_ch = [fig.add_subplot(gs_bottom[i, j]) for i in range(2) for j in range(2)]

    for ch in range(4):
        ax_ch[ch].plot(X_train[chosen_outlier_idx, :, ch], color=f"C{ch}")
        ax_ch[ch].set_title(f"Channel {ch}")
        ax_ch[ch].set_xlabel("Time")
        ax_ch[ch].set_ylabel("Amplitude")

    con1 = ConnectionPatch(xyA=(chosen_outlier[-1], chosen_outlier[-2]), xyB=(-0.25, 0.75),
                           coordsA="data", coordsB="axes fraction",
                           axesA=ax, axesB=ax_signal,
                           color="black", linewidth=1.5, linestyle="dashed", zorder=1)
    fig.patches.append(con1)

    con2 = ConnectionPatch(xyA=(chosen_outlier[-1], chosen_outlier[-2]), xyB=(-0.25, 0.05),
                           coordsA="data", coordsB="axes fraction",
                           axesA=ax, axesB=ax_ch[0],
                           color="black", linewidth=1.5, linestyle="dashed", zorder=1)
    fig.patches.append(con2)

    ax.scatter(chosen_outlier[-2], chosen_outlier[-1], s=30,
               c=[chosen_color], marker='o', edgecolor='black', linewidth=1.2, zorder=3)

    plt.tight_layout()
    filename = 'CFA Zoomed In Outlier.png'
    plt.savefig(filename)
    files.download(filename)
    plt.show()


outlier_indices = identify_outliers_with_percentages(z_mean_1, Y_train, 3.0)[1]
# Fe3O4
# chosen_outlier_idx = outlier_indices[905]
# CFA
chosen_outlier_idx = outlier_indices[680]

plot_latent_space_with_signal(3.0, chosen_outlier_idx)


In [ ]:
#2D Channel 0

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from google.colab import files

z_mean, z_sd = vae.encode(X_train_C0)

target_names = ["FS", "FSglyc", "CFA", "ATD", "VA", "Fe2O3", "Fe3O4"]
cmap1 = ListedColormap(['red', 'purple', 'yellow', 'gray', 'brown', 'blue', 'green'])

fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for idx_0 in range(7):
    class_indices = np.where(Y_train == idx_0)
    z_mean_class = z_mean[class_indices]

    ax = axs[idx_0 // 4, idx_0 % 4]
    ax.scatter(z_mean_class[:, -1], z_mean_class[:, -2], s=1, color=cmap1(idx_0), label=target_names[idx_0])
    ax.set_xlabel("$z_1$", fontsize=15, labelpad=10)
    ax.set_ylabel("$z_2$", fontsize=15, labelpad=10)
    ax.legend(fontsize=11, loc='upper left')
    ax.set_title(target_names[idx_0], fontsize=16)

ax_all = axs[1, 3]
for idx_0 in range(7):
    class_indices = np.where(Y_train == idx_0)
    z_mean_class = z_mean[class_indices]
    ax_all.scatter(z_mean_class[:, -1], z_mean_class[:, -2], s=1, color=cmap1(idx_0), label=target_names[idx_0])

ax_all.set_xlabel("$z_1$", fontsize=15, labelpad=10)
ax_all.set_ylabel("$z_2$", fontsize=15, labelpad=10)
ax_all.legend(fontsize=11, loc='upper left')
ax_all.set_title("All Classes", fontsize=16)

plt.tight_layout(pad=3.0)
filename = 'individualAerosolClass_Channel_0.png'
plt.savefig(filename, dpi=300)
plt.show()

# High Resolution Download
files.download(filename)



In [ ]:
#2D Channel 1

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from google.colab import files

z_mean_1, z_sd_1 = vae.encode(X_train_C1)

target_names = ["FS", "FSglyc", "CFA", "ATD", "VA", "Fe2O3", "Fe3O4"]
cmap1 = ListedColormap(['red', 'purple', 'yellow', 'gray', 'brown', 'blue', 'green'])

fig, axs = plt.subplots(2, 4, figsize=(20, 10))

for idx_1 in range(7):
    class_indices = np.where(Y_train == idx_1)
    z_mean_class = z_mean_1[class_indices]

    ax = axs[idx_1 // 4, idx_1 % 4]
    ax.scatter(z_mean_class[:, -1], z_mean_class[:, -2], s=1, color=cmap1(idx_1), label=target_names[idx_1])
    ax.set_xlabel("$z_3$", fontsize=15, labelpad=10)
    ax.set_ylabel("$z_4$", fontsize=15, labelpad=10)
    ax.legend(fontsize=11, loc='upper left')
    ax.set_title(target_names[idx_1], fontsize=16)
    ax.grid(False)

# For the combined plot
ax_all = axs[1, 3]
for idx_1 in range(7):
    class_indices = np.where(Y_train == idx_1)
    z_mean_class = z_mean_1[class_indices]
    ax_all.scatter(z_mean_class[:, -1], z_mean_class[:, -2], s=1, color=cmap1(idx_1), label=target_names[idx_1])

ax_all.set_xlabel("$z_3$", fontsize=15, labelpad=10)
ax_all.set_ylabel("$z_4$", fontsize=15, labelpad=10)
ax_all.legend(fontsize=11, loc='upper left')
ax_all.set_title("All Classes", fontsize=16)
ax.grid(False)

plt.tight_layout(pad=3.0)
filename = 'individualAerosolClass_Channel_1.png'
plt.savefig(filename, dpi=300)
plt.show()

# High Resolution Download
files.download(filename)

In [ ]:
X_train_C0.shape

torch.Size([85832, 400])

In [ ]:
X_train.shape

(85832, 400, 4)

In [ ]:
original_max_X_train.shape

(85832, 4)

In [ ]:
incandescent_max = []
incandescent_max = original_max_X_train[:, 1]

new_incandescent_max = np.expand_dims(incandescent_max, axis = 1)
new_incandescent_max.shape

(85832, 1)

In [ ]:
X_train.shape

(85832, 400, 4)

In [ ]:
scattering = []
scattering = original_max_X_train[:, 0]

new_scattering = np.expand_dims(scattering, axis = 1)
new_scattering.shape

(85832, 1)

In [ ]:
z_mean.shape

torch.Size([85832, 2])

In [ ]:
z_mean_1.shape

torch.Size([85832, 2])

In [ ]:
print(z_mean.numpy().shape)
print(z_mean_1.numpy().shape)
print(new_scattering.shape)
print(new_incandescent_max.shape)
print(color_channel_ratio.shape)


(85832, 2)
(85832, 2)
(85832, 1)
(85832, 1)
(85832,)


In [ ]:
color_channel_ratio.shape
color_channel_ratio = np.array(color_channel_ratio)
color_channel_ratio = np.expand_dims(color_channel_ratio, axis = 1)

color_channel_ratio.shape

(85832, 1)

In [ ]:
np.count_nonzero(np.isinf(color_channel_ratio))
bad_data = np.isinf(color_channel_ratio)
bad_data = bad_data[:, 0]
bad_data.shape

(85832,)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X_all = np.concatenate((z_mean.numpy(), z_mean_1.numpy(), new_scattering, new_incandescent_max, color_channel_ratio), axis = 1)
#X_all = np.concatenate (first two pca components ch. 0, first two pca components ch. 1, scattering, incades)
#pass above line into random forest, and compare it with old random forest to see which is doing better. gives us more convincing analysis
X_filtered = X_all[~bad_data,:]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_filtered)
X_scaled.shape

(85832, 7)

In [ ]:
Y_filtered = Y_train[~bad_data]
Y_filtered
Y_filtered.shape

(85832,)

In [ ]:
z_mean.shape

torch.Size([85832, 2])

In [ ]:
z_mean_1.shape

torch.Size([85832, 2])

In [ ]:
new_scattering.shape

(85832, 1)

In [ ]:
new_scattering

array([[0.9186803 ],
       [0.04637546],
       [0.00833333],
       ...,
       [2.61570632],
       [2.61654275],
       [2.61728625]])

In [ ]:
new_incandescent_max

array([[13.22634017],
       [ 1.65205162],
       [ 0.43266049],
       ...,
       [23.28590338],
       [ 9.48792191],
       [23.54649239]])

In [ ]:
color_channel_ratio

array([[1.11002002],
       [1.08262394],
       [1.21032505],
       ...,
       [1.52508171],
       [1.49707908],
       [1.61149563]])

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

n_clusters = 7
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X_scaled)

kmeans_labels = kmeans.labels_
kmeans_centers = kmeans.cluster_centers_

data = pd.DataFrame(X_scaled, columns=['z1', 'z2', 'z3', 'z4', 'scattering_max', 'incandescent_max', 'color_ratio'])
data['Cluster'] = kmeans_labels

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 8), dpi=300)
sns.scatterplot(data=data, x='color_ratio', y='incandescent_max', hue='Cluster', palette='viridis', alpha=0.6)

for center in kmeans_centers:
    plt.scatter(center[4], center[3], s=200, c='red', marker='X')  # Mark cluster centers

plt.title('K-means Clustering in 2D Latent Space')
plt.xlabel('Color Ratio')
plt.ylabel('Incandescent Max')
plt.xlim(0, 0.2)
plt.legend(title='Cluster')

plt.savefig('kmeans_clustering_high_res.png', dpi=300)

plt.show()

from google.colab import files
files.download('kmeans_clustering_high_res.png')


In [ ]:
import matplotlib.pyplot as plt

# Plot for 'z1'
plt.figure(figsize=(5, 4), dpi=300)
plt.xlim(0, 0.2)
sc = plt.scatter(data['color_ratio'], data['incandescent_max'], c=data['z1'], s=1, cmap='viridis', alpha=0.6)
plt.xlabel("Color Ratio (Arbitrary Units)")
plt.ylabel("Incandescent Maximum (Normalized)")
cbar = plt.colorbar(sc)
cbar.set_label("z1")
plt.savefig('scatter_plot_z1_high_res.png', dpi=300)
plt.show()

# Plot for 'z2'
plt.figure(figsize=(5, 4), dpi=300)
plt.xlim(0, 0.2)
sc = plt.scatter(data['color_ratio'], data['incandescent_max'], c=data['z2'], s=1, cmap='viridis', alpha=0.6)
plt.xlabel("Color Ratio (Arbitrary Units)")
plt.ylabel("Incandescent Maximum (Normalized)")
cbar = plt.colorbar(sc)
cbar.set_label("z2")
plt.savefig('scatter_plot_z2_high_res.png', dpi=300)
plt.show()

# Plot for 'z3'
plt.figure(figsize=(5, 4), dpi=300)
plt.xlim(0, 0.2)
sc = plt.scatter(data['color_ratio'], data['incandescent_max'], c=data['z3'], s=1, cmap='viridis', alpha=0.6)
plt.xlabel("Color Ratio (Arbitrary Units)")
plt.ylabel("Incandescent Maximum (Normalized)")
cbar = plt.colorbar(sc)
cbar.set_label("z3")
plt.savefig('scatter_plot_z3_high_res.png', dpi=300)
plt.show()

# Plot for 'z4'
plt.figure(figsize=(5, 4), dpi=300)
plt.xlim(0, 0.2)
sc = plt.scatter(data['color_ratio'], data['incandescent_max'], c=data['z4'], s=1, cmap='viridis', alpha=0.6)
plt.xlabel("Color Ratio (Arbitrary Units)")
plt.ylabel("Incandescent Maximum (Normalized)")
cbar = plt.colorbar(sc)
cbar.set_label("z4")
plt.savefig('scatter_plot_z4_high_res.png', dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from google.colab import files

fig, axes = plt.subplots(2, 2, figsize=(10, 8), dpi=300)
fig.subplots_adjust(hspace=0.3, wspace=0.3)

z_data = [data['z1'], data['z2'], data['z3'], data['z4']]
z_labels = ['z1', 'z2', 'z3', 'z4']

axes = axes.flatten()

for i in range(4):
    sc = axes[i].scatter(
        data['color_ratio'],
        data['incandescent_max'],
        c=z_data[i],
        s=1,
        cmap='viridis',
        alpha=0.6
    )
    axes[i].set_xlim(0, 0.2)
    axes[i].set_xlabel("Color Ratio (Arbitrary Units)")
    axes[i].set_ylabel("Incandescent Maximum (Normalized)")
    cbar = fig.colorbar(sc, ax=axes[i])
    cbar.set_label(z_labels[i])
    # axes[i].set_title(f"Scatter Plot Colored by {z_labels[i]}")

plt.tight_layout()
plt.savefig("color_ratio_z1_to_z4.png", dpi=300)
plt.show()


files.download("color_ratio_z1_to_z4.png")


In [ ]:
#RANDOM FOREST

from sklearn.model_selection import train_test_split
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_scaled, Y_filtered, test_size=0.33, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train_rf, y_train_rf)

RandomForestClassifier()

In [ ]:
y_output = rf.predict(X_test_rf)

In [ ]:
# RANDOM FOREST Confusion matrix

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files

target_names = ["FS", "FSglyc", "CFA", "ATD", "VA", "Fe2O3", "Fe3O4"]
correlation_arr = confusion_matrix(y_output, y_test_rf, normalize='true')

plt.figure(figsize=(10, 8), dpi=300)
ax = sns.heatmap(
    correlation_arr,
    annot=True,
    fmt=".2%",
    cmap='viridis',
    xticklabels=target_names,
    yticklabels=target_names
)
ax.set_title('Confusion Matrix with Normalized Percentages')
ax.set_ylabel('Predicted Label')
ax.set_xlabel('True Label')

plt.savefig('confusion_matrix_high_res.png', dpi=300)
plt.show()
files.download('confusion_matrix_high_res.png')
